# Getting netCDF4 Data

Because of Github's file size limit, we couldn't upload this part of tha data; So, the bellow cell download it direcly from our font.

In [2]:
# !wget ftp://ftp.cdc.noaa.gov/Datasets/icoads/2degree/enh/sst.mean.nc
# !wget ftp://ftp.cdc.noaa.gov/Datasets/icoads/2degree/enh/rhum.mean.nc
# !wget ftp://ftp.cdc.noaa.gov/Datasets/icoads/2degree/enh/slp.mean.nc
# !wget ftp://ftp.cdc.noaa.gov/Datasets/icoads/2degree/enh/vwnd.mean.nc
# !wget ftp://ftp.cdc.noaa.gov/Datasets/icoads/2degree/enh/wspd.mean.nc
# !wget ftp://ftp.cdc.noaa.gov/Datasets/icoads/2degree/enh/cldc.mean.nc
# !wget ftp://ftp.cdc.noaa.gov/Datasets/icoads/2degree/enh/air.mean.nc
# !wget ftp://ftp.cdc.noaa.gov/Datasets/icoads/2degree/enh/uwnd.mean.nc
# !wget ftp://ftp.cdc.noaa.gov/Datasets/icoads/2degree/enh/sflx.mean.nc

# Cleaning Hurricane Datasets

In [135]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import netCDF4 as nc
import datetime as dt
%matplotlib inline

In [142]:
df = pd.read_csv('data_atl_filtrado.csv')

df.head()

,ID,Name,Date,Time,Event,Status,Latitude,Longitude,Maximum Wind,Minimum Pressure,...,Moderate Wind NW,High Wind NE,High Wind SE,High Wind SW,High Wind NW,Date_c,Year,Month,Day,Duration
0,AL011951,UNNAMED,19510104,1800,,TS,24.0N,58.9W,45,-999,...,-999,-999,-999,-999,-999,1951-01-04,1951,1,4,5
1,AL011951,UNNAMED,19510105,0,,TS,22.7N,60.0W,45,-999,...,-999,-999,-999,-999,-999,1951-01-05,1951,1,5,5
2,AL011951,UNNAMED,19510105,600,,TS,22.1N,60.7W,45,-999,...,-999,-999,-999,-999,-999,1951-01-05,1951,1,5,5
3,AL011951,UNNAMED,19510105,1200,,TS,21.7N,61.1W,45,-999,...,-999,-999,-999,-999,-999,1951-01-05,1951,1,5,5
4,AL011951,UNNAMED,19510105,1800,,TS,21.5N,61.3W,45,-999,...,-999,-999,-999,-999,-999,1951-01-05,1951,1,5,5


In [143]:
# Formatting latitude and longitude to floats
df.Date = pd.to_datetime(df.Date, format="%Y%m%d")
# df["month"] = df["Date"].apply(lambda x: x.month)
# df["year"] = df["Date"].apply(lambda x: x.year)
df.Latitude = df.Latitude.apply(lambda x: -float(x.rstrip("S")) if x.endswith("S") else float(x.rstrip("N")))
df.Longitude = df.Longitude.apply(lambda x: -float(x.rstrip("W")) if x.endswith("W") else float(x.rstrip("E")))

# df2.Date = pd.to_datetime(df2.Date, format="%Y%m%d")
# df2["month"] = df2["Date"].apply(lambda x: x.month)
# df2["year"] = df2["Date"].apply(lambda x: x.year)
# df2.Latitude = df2.Latitude.apply(lambda x: -float(x.rstrip("S")) if x.endswith("S") else float(x.rstrip("N")))
# df2.Longitude = df2.Longitude.apply(lambda x: -float(x.rstrip("W")) if x.endswith("W") else float(x.rstrip("E")))


In [144]:
df.dtypes

ID                          object
Name                        object
Date                datetime64[ns]
Time                         int64
Event                       object
Status                      object
Latitude                   float64
Longitude                  float64
Maximum Wind                 int64
Minimum Pressure             int64
Low Wind NE                  int64
Low Wind SE                  int64
Low Wind SW                  int64
Low Wind NW                  int64
Moderate Wind NE             int64
Moderate Wind SE             int64
Moderate Wind SW             int64
Moderate Wind NW             int64
High Wind NE                 int64
High Wind SE                 int64
High Wind SW                 int64
High Wind NW                 int64
Date_c                      object
Year                         int64
Month                        int64
Day                          int64
Duration                     int64
dtype: object

In [145]:
df[['Latitude','Longitude']].describe()

,Latitude,Longitude
count,5544.000000,5544.000000
mean,17.943038,-62.555538
std,3.761142,18.587708
min,8.800000,-104.300000
25%,15.000000,-78.200000
50%,17.800000,-61.900000
75%,21.000000,-48.100000
max,24.900000,-18.400000


In [146]:
#Text entries are spaced
np.unique(df.Name)[0:5],np.unique(df.Event),np.unique(df.Status)

(array(['                ANA', '                DOG',
        '                FAY', '                FOX',
        '                IDA'], dtype=object),
 array(['  ', ' C', ' I', ' L', ' P', ' R', ' S', ' T', ' W'], dtype=object),
 array([' DB', ' EX', ' HU', ' LO', ' SS', ' TS', ' WV'], dtype=object))

In [147]:
#Formatting text entries

df['Name'] = df['Name'].apply(lambda x: x.strip())
df['Event'] = df['Event'].apply(lambda x: x.strip())
df['Status'] = df['Status'].apply(lambda x: x.strip())

df2['Name'] = df2['Name'].apply(lambda x: x.strip())
df2['Event'] = df2['Event'].apply(lambda x: x.strip())
df2['Status'] = df2['Status'].apply(lambda x: x.strip())

np.unique(df.Name)[0:5],np.unique(df.Event),np.unique(df.Status)

(array(['ABBY', 'ABLE', 'ALBERTO', 'ALEX', 'ALICE'], dtype=object),
 array(['', 'C', 'I', 'L', 'P', 'R', 'S', 'T', 'W'], dtype=object),
 array(['DB', 'EX', 'HU', 'LO', 'SS', 'TS', 'WV'], dtype=object))

In [148]:
pd.DataFrame(zip(df.columns,
                 [np.sum(df[x] == -999)/len(df) for x in df.columns])
             ,columns = ['Variable','Missing Ratio'])

,Variable,Missing Ratio
0,ID,0.000000
1,Name,0.000000
2,Date,0.000000
3,Time,0.000000
4,Event,0.000000
5,Status,0.000000
6,Latitude,0.000000
7,Longitude,0.000000
8,Maximum Wind,0.000000
9,Minimum Pressure,0.209416


In [50]:
# df = df.drop(['Low Wind NE',
#        'Low Wind SE', 'Low Wind SW', 'Low Wind NW', 'Moderate Wind NE',
#        'Moderate Wind SE', 'Moderate Wind SW', 'Moderate Wind NW',
#        'High Wind NE', 'High Wind SE', 'High Wind SW', 'High Wind NW'],axis = 1)

In [149]:
sst_mean = nc.Dataset('sst.mean.nc','r')
rhum_mean = nc.Dataset('rhum.mean.nc','r')
wspd_mean = nc.Dataset('wspd.mean.nc','r')
slp_mean = nc.Dataset('slp.mean.nc','r')
# vwnd_mean = nc.Dataset('Datasets/vwnd.mean.nc','r')
cldc_mean = nc.Dataset("cldc.mean.nc",'r')

lats = sst_mean.variables['lat'][:]
lons = sst_mean.variables['lon'][:]
time = sst_mean.variables['time'][:]

lons = [x-180 for x in lons]
lats = [x for x in lats]

sst = sst_mean.variables['sst'][:,:,:]
rhum = rhum_mean.variables['rhum'][:,:,:]
wspd = wspd_mean.variables['wspd'][:,:,:]
slp = slp_mean.variables['slp'][:,:,:]
# vwnd = vwnd_mean.variables['vwnd'][:,:,:]
cldc = cldc_mean.variables['cldc'][:,:,:]

sst_mean.close()
wspd_mean.close()
rhum_mean.close()
slp_mean.close()
# vwnd_mean.close()
cldc_mean.close()

In [150]:
from math import sin, cos, sqrt, atan2, radians
def distance(lat1,lon1,lat2,lon2):
    # approximate radius of earth in km
    R = 6371.0

    lat1 = radians(lat1)
    lon1 = radians(lon1)
    lat2 = radians(lat2)
    lon2 = radians(lon2)

    dlon = lon2 - lon1
    dlat = lat2 - lat1

    a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))
    if R*c==0:
        return 0.0000001 #Handling Zero Division problems
    else:
        return R * c

In [151]:
def get_coord(coord,l,prob):
    if int(coord) in l:
        return int(coord)
    elif int(coord)+1 in l:
        return int(coord)+1
    elif int(coord)-1 in l:
        return int(coord)-1
    elif int(coord)+2 in l:
        return int(coord)+2
    elif int(coord)-2 in l:
        return int(coord)-2
    
def get_neighboors(t,lat,lon,marray,k):
    nb = []
    dist = []
    prob = []
    lat_data = get_coord(lat,lats,prob) #nearest lat in our lats list
    lon_data = get_coord(lon,lons,prob)
    lat_i = lats.index(lat_data)
    lon_i = lons.index(lon_data)
    if marray[t,lat_i,lon_i]:
        nb.append(marray[t,lat_i,lon_i])
        dist.append(distance(lat,lon,lat_data,lon_data))
    j=1
    while len(nb)<k:
        
        lower_i = lat_i-j
        upper_i = lat_i+j
        right_i = lon_i+j
        left_i = lon_i-j
#         if right_i>=len()
        left_values = marray[t,lower_i:upper_i,left_i]
        upper_values = marray[t,upper_i,left_i:right_i]
        right_values = marray[t,upper_i:lower_i:-1,right_i]
        lower_values = marray[t,lower_i,right_i:left_i:-1]
        
        [nb.append(x) for x in left_values if x]
        [dist.append(distance(lat,lon,lats[i],lons[left_i])) for i in range(len(left_values)) if left_values[i]]
        if len(nb)>=k:
            break
        [nb.append(x) for x in upper_values if x]
        [dist.append(distance(lat,lon,lats[upper_i],lons[i])) for i in range(len(upper_values)) if upper_values[i]]
        if len(nb)>=k:
            break
        [nb.append(x) for x in right_values if x]
        [dist.append(distance(lat,lon,lats[i],lons[right_i])) for i in range(len(right_values)) if right_values[i]]
        if len(nb)>=k:
            break
        [nb.append(x) for x in lower_values if x]
        [dist.append(distance(lat,lon,lats[lower_i],lons[i])) for i in range(len(lower_values)) if lower_values[i]]
        if len(nb)>=k:
            break
        j+=1
    if prob!=[]:
        print(prob)
    return nb,dist
        
        

In [152]:
period = pd.date_range(start = "1800-01-01",end = "2020-07-01", freq = "MS").to_pydatetime().tolist()
def get_data(datetime,lat,lon,dataset,k):#k is the number of neighboors
    year = datetime.year
    month = datetime.month
    time_index = period.index(dt.datetime(year, month, 1, 0, 0))
    nb,dist = get_neighboors(time_index,lat,lon,dataset,k)
    inv_dist = [1/x for x in dist]
    return sum(nb[i]*inv_dist[i] for i in range(len(nb)))/sum(inv_dist)

In [153]:
def get_data_df(df,dataset,label,k=15):
    output = pd.DataFrame(np.zeros([len(df),1]),columns = [label])
    for i in range(len(df)):
        output.loc[i,label] = get_data(df.Date[i],df.Latitude[i],df.Longitude[i],dataset,k)
    return output

In [154]:
df.loc[:,'sst']=get_data_df(df,sst,'sst')
df.loc[:,'rhum']=get_data_df(df,rhum,'rhum')
df.loc[:,'wspd']=get_data_df(df,rhum,'wspd')
df.loc[:,'slp']=get_data_df(df,slp,'slp')
df.loc[:,'cldc']=get_data_df(df,cldc,'cldc')
df.columns

Index(['ID', 'Name', 'Date', 'Time', 'Event', 'Status', 'Latitude',
       'Longitude', 'Maximum Wind', 'Minimum Pressure', 'Low Wind NE',
       'Low Wind SE', 'Low Wind SW', 'Low Wind NW', 'Moderate Wind NE',
       'Moderate Wind SE', 'Moderate Wind SW', 'Moderate Wind NW',
       'High Wind NE', 'High Wind SE', 'High Wind SW', 'High Wind NW',
       'Date_c', 'Year', 'Month', 'Day', 'Duration', 'sst', 'rhum', 'wspd',
       'slp', 'cldc'],
      dtype='object')

In [155]:
df = df[['ID', 'Name', 'Date','Year','Month','Day', 'Time', 'Event', 'Status', 'Latitude',
       'Longitude', 'Maximum Wind', 'Minimum Pressure', 'Low Wind NE',
       'Low Wind SE', 'Low Wind SW', 'Low Wind NW', 'Moderate Wind NE',
       'Moderate Wind SE', 'Moderate Wind SW', 'Moderate Wind NW',
       'High Wind NE', 'High Wind SE', 'High Wind SW', 'High Wind NW',
       'Date_c', 'Duration', 'sst',
       'rhum', 'wspd', 'slp', 'cldc']]
df.columns

Index(['ID', 'Name', 'Date', 'Year', 'Month', 'Day', 'Time', 'Event', 'Status',
       'Latitude', 'Longitude', 'Maximum Wind', 'Minimum Pressure',
       'Low Wind NE', 'Low Wind SE', 'Low Wind SW', 'Low Wind NW',
       'Moderate Wind NE', 'Moderate Wind SE', 'Moderate Wind SW',
       'Moderate Wind NW', 'High Wind NE', 'High Wind SE', 'High Wind SW',
       'High Wind NW', 'Date_c', 'Duration', 'sst', 'rhum', 'wspd', 'slp',
       'cldc'],
      dtype='object')

In [157]:
df.to_csv('data_atl_merged.csv')